# Кросс-валидация (Cross-Validation)

## Введение

Кросс-валидация — это метод оценки качества модели машинного обучения на ограниченных данных. Основная идея: разделить данные на несколько частей (фолдов), обучить модель на одних частях и протестировать на других, повторяя процесс для всех комбинаций.

### Применение в биологии:
- Оценка моделей классификации заболеваний
- Предсказание эффективности лекарств
- Анализ экспрессии генов
- Валидация биомаркеров

### Зачем нужна кросс-валидация?
- Более надежная оценка качества модели
- Защита от переобучения
- Эффективное использование ограниченных данных
- Выбор гиперпараметров

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (
    KFold, StratifiedKFold, LeaveOneOut, ShuffleSplit,
    cross_val_score, cross_validate, learning_curve,
    GridSearchCV, RandomizedSearchCV
)
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
import pandas as pd

np.random.seed(42)
sns.set_style('whitegrid')

## 1. Проблема: почему простое разбиение Train/Test недостаточно?

In [ ]:
from sklearn.model_selection import train_test_split

# Генерируем данные: классификация типов клеток
X, y = make_classification(
    n_samples=100,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

# Демонстрируем вариабельность при разных разбиениях
n_splits = 10
accuracies = []

for i in range(n_splits):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=i
    )
    
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    accuracies.append(accuracy)

print(f"Accuracies для разных разбиений: {[f'{acc:.3f}' for acc in accuracies]}")
print(f"\nСреднее: {np.mean(accuracies):.3f}")
print(f"Стандартное отклонение: {np.std(accuracies):.3f}")
print(f"Разброс: {np.max(accuracies) - np.min(accuracies):.3f}")

# Визуализация
plt.figure(figsize=(10, 5))
plt.bar(range(n_splits), accuracies, alpha=0.7, color='steelblue')
plt.axhline(np.mean(accuracies), color='red', linestyle='--', linewidth=2, label=f'Среднее: {np.mean(accuracies):.3f}')
plt.xlabel('Номер разбиения')
plt.ylabel('Accuracy')
plt.title('Вариабельность accuracy при разных Train/Test разбиениях')
plt.legend()
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("\n⚠️ Проблема: результат сильно зависит от случайного разбиения!")

## 2. K-Fold Cross-Validation

Самый популярный метод кросс-валидации:
1. Разделить данные на K частей (фолдов)
2. Для каждого фолда:
   - Использовать его как тестовую выборку
   - Остальные K-1 фолдов — как обучающую
3. Усреднить результаты

In [ ]:
# Визуализация K-Fold разбиения
def visualize_cv_split(cv, X, y, title):
    """Визуализация разбиения данных при кросс-валидации"""
    fig, axes = plt.subplots(len(list(cv.split(X, y))), 1, figsize=(12, len(list(cv.split(X, y))) * 0.8))
    
    if len(list(cv.split(X, y))) == 1:
        axes = [axes]
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        # Создаем массив для визуализации
        indices = np.arange(len(X))
        colors = np.array(['white'] * len(X))
        colors[train_idx] = 'blue'
        colors[test_idx] = 'red'
        
        # Рисуем
        axes[idx].scatter(indices, [idx] * len(indices), c=colors, marker='s', s=50, edgecolors='black', linewidths=0.5)
        axes[idx].set_yticks([idx])
        axes[idx].set_yticklabels([f'Fold {idx + 1}'])
        axes[idx].set_xlim(-1, len(X))
        axes[idx].set_ylim(idx - 0.5, idx + 0.5)
        
        # Аннотация
        axes[idx].text(len(X) + 5, idx, f'Train: {len(train_idx)}  Test: {len(test_idx)}', 
                      verticalalignment='center', fontsize=9)
    
    axes[-1].set_xlabel('Индекс образца')
    fig.suptitle(title, fontsize=14, fontweight='bold', y=0.98)
    
    # Легенда
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', edgecolor='black', label='Обучающая выборка'),
        Patch(facecolor='red', edgecolor='black', label='Тестовая выборка')
    ]
    axes[0].legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(1.2, 1.3))
    
    plt.tight_layout()
    plt.show()

# Создаем небольшой датасет для визуализации
X_small = np.arange(50).reshape(-1, 1)
y_small = np.random.randint(0, 2, 50)

# 5-Fold CV
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
visualize_cv_split(kfold, X_small, y_small, '5-Fold Cross-Validation')

In [ ]:
# Применяем K-Fold CV
model = LogisticRegression(max_iter=1000)

# 5-fold CV
cv_scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')

print("5-Fold Cross-Validation:")
print(f"Scores по фолдам: {[f'{score:.3f}' for score in cv_scores]}")
print(f"\nСредний accuracy: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")

# Сравниваем с одним разбиением
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model.fit(X_train, y_train)
single_split_score = model.score(X_test, y_test)

print(f"\nОдно разбиение train/test: {single_split_score:.3f}")
print(f"\n✅ Кросс-валидация дает более надежную оценку!")

## 3. Типы кросс-валидации

### 3.1 Stratified K-Fold
Сохраняет пропорции классов в каждом фолде (важно для несбалансированных данных)

In [ ]:
# Создаем несбалансированный датасет
X_imb, y_imb = make_classification(
    n_samples=100,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    weights=[0.8, 0.2],  # 80% класс 0, 20% класс 1
    random_state=42
)

print(f"Распределение классов: {np.bincount(y_imb)}")
print(f"Процентное соотношение: {np.bincount(y_imb) / len(y_imb) * 100}")

# Сравниваем обычный KFold и Stratified KFold
print("\n" + "="*60)
print("Обычный K-Fold:")
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(kfold.split(X_imb, y_imb), 1):
    print(f"Fold {fold}: Test set class distribution: {np.bincount(y_imb[test_idx])}")

print("\n" + "="*60)
print("Stratified K-Fold:")
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, test_idx) in enumerate(skfold.split(X_imb, y_imb), 1):
    print(f"Fold {fold}: Test set class distribution: {np.bincount(y_imb[test_idx])}")

print("\n✅ Stratified K-Fold сохраняет пропорции классов!")

In [ ]:
# Сравниваем качество оценки
cv_regular = cross_val_score(model, X_imb, y_imb, cv=KFold(n_splits=5, shuffle=True, random_state=42))
cv_stratified = cross_val_score(model, X_imb, y_imb, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].bar(range(5), cv_regular, alpha=0.7, color='steelblue', label='Regular K-Fold')
axes[0].axhline(cv_regular.mean(), color='red', linestyle='--', linewidth=2)
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Accuracy')
axes[0].set_title(f'Regular K-Fold\nMean: {cv_regular.mean():.3f} ± {cv_regular.std():.3f}')
axes[0].set_ylim([0.7, 1.0])
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].bar(range(5), cv_stratified, alpha=0.7, color='seagreen', label='Stratified K-Fold')
axes[1].axhline(cv_stratified.mean(), color='red', linestyle='--', linewidth=2)
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Accuracy')
axes[1].set_title(f'Stratified K-Fold\nMean: {cv_stratified.mean():.3f} ± {cv_stratified.std():.3f}')
axes[1].set_ylim([0.7, 1.0])
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

### 3.2 Leave-One-Out Cross-Validation (LOOCV)
K = n (количество образцов). Каждый образец по очереди становится тестовым.

In [ ]:
# LOOCV на небольшом датасете
X_small_data, y_small_data = make_classification(n_samples=30, n_features=5, n_informative=3, random_state=42)

loo = LeaveOneOut()
cv_loo = cross_val_score(model, X_small_data, y_small_data, cv=loo)

print(f"LOOCV результаты:")
print(f"Количество фолдов: {loo.get_n_splits(X_small_data)}")
print(f"Средний accuracy: {cv_loo.mean():.3f}")
print(f"\n⚠️ LOOCV очень затратный по времени для больших датасетов!")
print(f"   Для датасета из 1000 образцов потребуется 1000 обучений модели.")

### 3.3 Shuffle Split
Случайные разбиения на train/test с заданными пропорциями

In [ ]:
# Shuffle Split
shuffle_split = ShuffleSplit(n_splits=10, test_size=0.3, random_state=42)
cv_shuffle = cross_val_score(model, X, y, cv=shuffle_split)

print(f"Shuffle Split (10 итераций, 30% test):")
print(f"Средний accuracy: {cv_shuffle.mean():.3f} ± {cv_shuffle.std():.3f}")

# Визуализация
visualize_cv_split(ShuffleSplit(n_splits=5, test_size=0.3, random_state=42), 
                   X_small, y_small, 'Shuffle Split (5 splits, 30% test)')

## 4. Детальный анализ с cross_validate

Функция `cross_validate` позволяет получить несколько метрик одновременно

In [ ]:
# Множественные метрики
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}

cv_results = cross_validate(
    model, X, y, 
    cv=5, 
    scoring=scoring,
    return_train_score=True
)

# Создаем DataFrame для удобства
results_df = pd.DataFrame({
    'Fold': range(1, 6),
    'Train Accuracy': cv_results['train_accuracy'],
    'Test Accuracy': cv_results['test_accuracy'],
    'Test Precision': cv_results['test_precision'],
    'Test Recall': cv_results['test_recall'],
    'Test F1': cv_results['test_f1']
})

print("Детальные результаты кросс-валидации:")
print(results_df.to_string(index=False))
print("\nСредние значения:")
print(results_df.mean(numeric_only=True).to_string())

In [ ]:
# Визуализация метрик
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# График 1: Все метрики по фолдам
metrics = ['Test Accuracy', 'Test Precision', 'Test Recall', 'Test F1']
x = np.arange(5)
width = 0.2

for i, metric in enumerate(metrics):
    axes[0].bar(x + i*width, results_df[metric], width, label=metric.replace('Test ', ''), alpha=0.8)

axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Score')
axes[0].set_title('Метрики по фолдам')
axes[0].set_xticks(x + width * 1.5)
axes[0].set_xticklabels([f'Fold {i+1}' for i in range(5)])
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# График 2: Train vs Test Accuracy
axes[1].plot(range(1, 6), results_df['Train Accuracy'], 'o-', linewidth=2, markersize=8, label='Train Accuracy')
axes[1].plot(range(1, 6), results_df['Test Accuracy'], 's-', linewidth=2, markersize=8, label='Test Accuracy')
axes[1].fill_between(range(1, 6), results_df['Train Accuracy'], results_df['Test Accuracy'], alpha=0.2)
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Сравнение Train и Test Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Learning Curves — диагностика модели

Learning curves показывают, как меняется качество модели в зависимости от размера обучающей выборки

In [ ]:
# Генерируем больший датасет
X_large, y_large = make_classification(
    n_samples=500,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

# Строим learning curves для разных моделей
models_to_compare = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42)
}

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, (name, model) in enumerate(models_to_compare.items()):
    train_sizes, train_scores, test_scores = learning_curve(
        model, X_large, y_large,
        cv=5,
        n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10),
        random_state=42
    )
    
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    axes[idx].plot(train_sizes, train_mean, 'o-', linewidth=2, label='Train score')
    axes[idx].fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.2)
    
    axes[idx].plot(train_sizes, test_mean, 's-', linewidth=2, label='CV score')
    axes[idx].fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.2)
    
    axes[idx].set_xlabel('Размер обучающей выборки')
    axes[idx].set_ylabel('Accuracy')
    axes[idx].set_title(f'{name}\nLearning Curve')
    axes[idx].legend(loc='lower right')
    axes[idx].grid(True, alpha=0.3)
    axes[idx].set_ylim([0.5, 1.05])

plt.tight_layout()
plt.show()

## 6. Подбор гиперпараметров с Grid Search

Кросс-валидация — ключевой инструмент для подбора гиперпараметров

In [ ]:
# Загружаем реальный биологический датасет
data = load_breast_cancer()
X_cancer = data.data
y_cancer = data.target

print("Датасет: Wisconsin Breast Cancer")
print(f"Количество образцов: {len(X_cancer)}")
print(f"Количество признаков: {X_cancer.shape[1]}")
print(f"Классы: {data.target_names}")
print(f"Распределение классов: {np.bincount(y_cancer)}")

# Нормализация данных
scaler = StandardScaler()
X_cancer_scaled = scaler.fit_transform(X_cancer)

In [ ]:
# Grid Search для Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    rf, 
    param_grid, 
    cv=5, 
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

print("Запуск Grid Search...")
print(f"Всего комбинаций параметров: {len(param_grid['n_estimators']) * len(param_grid['max_depth']) * len(param_grid['min_samples_split']) * len(param_grid['min_samples_leaf'])}")

grid_search.fit(X_cancer_scaled, y_cancer)

print("\n" + "="*60)
print("Результаты Grid Search:")
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший CV score: {grid_search.best_score_:.4f}")

In [ ]:
# Визуализация результатов Grid Search
results = pd.DataFrame(grid_search.cv_results_)
top_10 = results.nsmallest(10, 'rank_test_score')[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']]

print("\nТоп-10 комбинаций параметров:")
for idx, row in top_10.iterrows():
    print(f"Rank {int(row['rank_test_score'])}: Score = {row['mean_test_score']:.4f} (±{row['std_test_score']:.4f})")
    print(f"  Параметры: {row['params']}")
    print()

In [ ]:
# Heatmap: влияние двух параметров
# Фиксируем некоторые параметры и смотрим на влияние n_estimators и max_depth
pivot_data = results[results['param_min_samples_split'] == 2][results['param_min_samples_leaf'] == 1]

if len(pivot_data) > 0:
    pivot_table = pivot_data.pivot_table(
        values='mean_test_score',
        index='param_max_depth',
        columns='param_n_estimators'
    )
    
    plt.figure(figsize=(10, 6))
    sns.heatmap(pivot_table, annot=True, fmt='.3f', cmap='YlGnBu', cbar_kws={'label': 'Mean CV Score'})
    plt.title('Grid Search: Влияние n_estimators и max_depth\n(min_samples_split=2, min_samples_leaf=1)')
    plt.xlabel('n_estimators')
    plt.ylabel('max_depth')
    plt.tight_layout()
    plt.show()

## 7. Randomized Search — более эффективная альтернатива

Для больших пространств параметров Randomized Search часто эффективнее

In [ ]:
from scipy.stats import randint, uniform

# Распределения параметров
param_distributions = {
    'n_estimators': randint(50, 300),
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': uniform(0.5, 0.5)  # от 0.5 до 1.0
}

random_search = RandomizedSearchCV(
    rf,
    param_distributions,
    n_iter=50,  # количество случайных комбинаций
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("Запуск Randomized Search (50 итераций)...")
random_search.fit(X_cancer_scaled, y_cancer)

print("\n" + "="*60)
print("Результаты Randomized Search:")
print(f"Лучшие параметры: {random_search.best_params_}")
print(f"Лучший CV score: {random_search.best_score_:.4f}")

print("\nСравнение Grid Search vs Randomized Search:")
print(f"Grid Search - лучший score: {grid_search.best_score_:.4f}")
print(f"Randomized Search - лучший score: {random_search.best_score_:.4f}")

## 8. Nested Cross-Validation

Для несмещенной оценки качества модели с подбором гиперпараметров используется вложенная кросс-валидация:
- Внешний цикл — для оценки качества
- Внутренний цикл — для подбора гиперпараметров

In [ ]:
# Nested CV
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Упрощенная сетка параметров для скорости
param_grid_simple = {
    'n_estimators': [50, 100],
    'max_depth': [5, 10, None]
}

nested_scores = []

print("Nested Cross-Validation:")
for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X_cancer_scaled, y_cancer), 1):
    X_train, X_test = X_cancer_scaled[train_idx], X_cancer_scaled[test_idx]
    y_train, y_test = y_cancer[train_idx], y_cancer[test_idx]
    
    # Внутренний цикл - подбор гиперпараметров
    clf = GridSearchCV(rf, param_grid_simple, cv=inner_cv, scoring='accuracy')
    clf.fit(X_train, y_train)
    
    # Оценка на тестовой выборке внешнего фолда
    score = clf.score(X_test, y_test)
    nested_scores.append(score)
    
    print(f"Fold {fold}: Test Score = {score:.4f}, Best Params = {clf.best_params_}")

print(f"\nНесмещенная оценка модели: {np.mean(nested_scores):.4f} ± {np.std(nested_scores):.4f}")

## 9. Практические рекомендации

### Выбор количества фолдов:
- **K=5**: быстро, хорошо для больших датасетов
- **K=10**: золотой стандарт, баланс между bias и variance
- **LOOCV**: максимальное использование данных, но медленно

### Когда использовать Stratified CV:
- Несбалансированные классы
- Малые датасеты
- Классификация

### Grid Search vs Randomized Search:
- **Grid Search**: для небольших пространств параметров
- **Randomized Search**: для больших пространств, когда нужна скорость

### Nested CV:
- Всегда используйте для финальной оценки модели с подбором гиперпараметров
- Дает несмещенную оценку качества

## 10. Задания для самостоятельной работы

1. Реализуйте Time Series Cross-Validation для временных рядов
2. Сравните различные стратегии кросс-валидации на реальных биологических данных
3. Реализуйте собственный cross-validator для специфических задач (например, группированная кросс-валидация)
4. Исследуйте влияние размера валидационной выборки на стабильность оценок